In [1]:
import pandas as pd
import statistics
import numpy as np
import haversine as hs
from haversine import Unit
import math
from datetime import datetime
from scipy.stats import entropy
from scipy.stats import norm, kurtosis
import mysql.connector as msql
from mysql.connector import Error

In [2]:
def shannon_entropy(hist):
    side, count = np.unique(hist, return_counts=True)
    dist = count/len(hist)
    entropy_value = entropy(dist)
    return entropy_value
  
def kurtosis_fisher(dist):
    return kurtosis(dist)

def kurtosis_pearson(dist):
    return kurtosis(dist, fisher=False)

In [3]:
def get_features_by_traj(uids): 
    # unique trajectories
    count = 1
    total = len(uids)

    for uid in uids:
        
        uid = uid[0]
        # filtering a single feature and trajectoy
        feature_query = "SELECT %s FROM trajectories.gpspoints_sample WHERE identifier = '%s';" % ('latitude',uid)        
        mycursor.execute(feature_query)
        lats = mycursor.fetchall()

        feature_query = "SELECT %s FROM trajectories.gpspoints_sample WHERE identifier = '%s';" % ('longitude',uid)        
        mycursor.execute(feature_query)
        lngs = mycursor.fetchall()
        
        feature_query = "SELECT %s FROM trajectories.gpspoints_sample WHERE identifier = '%s';" % ('timestamp',uid)        
        mycursor.execute(feature_query)
        times = mycursor.fetchall()
        
        feature_query = "SELECT %s FROM trajectories.gpspoints_sample WHERE identifier = '%s';" % ('transport_mode',uid)        
        mycursor.execute(feature_query)
        labels = mycursor.fetchall()
        
        # secondary values that will be used to calculate the features
        dists = []
        datetimes = []
        speeds = []
        accs = []
        jerks = []
        
        #############################################################################################
        if(len(lats)>=80):
            # for each trajectory, use primary values to calculate secondary
            for i in range(1,len(lats)):
                coord1 = (lats[i-1][0],lngs[i-1][0])
                coord2 = (lats[i][0],lngs[i][0])
                dists.append(hs.haversine(coord1,coord2,unit=Unit.METERS))

                start = times[i-1][0]
                end = times[i][0]
                datetimes.append(pd.Timedelta((end - start)).total_seconds())

                if datetimes[i-1] == 0 or i == 1:
                    speeds.append(0)
                    accs.append(0)
                    jerks.append(0)
                else:
                    speeds.append(dists[i-1]/datetimes[i-1])
                    accs.append(speeds[i-1]-speeds[i-2]/datetimes[i-1])
                    jerks.append(accs[i-1]-accs[i-2]/datetimes[i-1])

            # populate database
            row = (uid, max(dists), np.mean(dists), min(dists), statistics.variance(dists), shannon_entropy(dists), 
                   kurtosis_fisher(dists), kurtosis_pearson(dists),
                   max(speeds), np.mean(speeds), min(speeds), statistics.variance(speeds), shannon_entropy(speeds), 
                   kurtosis_fisher(speeds), kurtosis_pearson(speeds),
                   max(accs), np.mean(accs), min(accs), statistics.variance(accs), shannon_entropy(accs), 
                   kurtosis_fisher(accs), kurtosis_pearson(accs), 
                   max(jerks), np.mean(jerks), min(jerks), statistics.variance(jerks), shannon_entropy(jerks), 
                   kurtosis_fisher(jerks), kurtosis_pearson(jerks),labels[i][0])

            feature_query = '''INSERT INTO trajectories.features_sample VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'''        
            mycursor.execute(feature_query, row)
        
            count += 1

In [4]:
mydb = msql.connect(
  host="localhost",
  user="root",
  password="root",
  database="trajectories"
)
mycursor = mydb.cursor(buffered=True)

try:
    if mydb.is_connected():

        uids_query = """SELECT DISTINCT identifier FROM trajectories.gpspoints_sample;"""
        mycursor.execute(uids_query)
        uids = mycursor.fetchall()
        get_features_by_traj(uids)
        mydb.commit()

except Error as e:

    print("Error while connecting to gpspoints:", e)

finally:
    if mydb.is_connected():
        mycursor.close()
        mydb.close()
        print("MySQL connection is closed")


MySQL connection is closed
